In [2]:
#ONE PICTURE AT A TIME
# Importing necessary functions
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os

# Initialising the ImageDataGenerator class.
# We will pass in the augmentation parameters in the constructor.
datagen = ImageDataGenerator(
		rotation_range = 5,
		shear_range = 1,
		zoom_range = 0.2,
		rescale=1)

input_path = 'C:/Users/Camille/Downloads/CROPPED LABELED RESIZED PRES PICS/'
for filename in os.listdir(input_path):
	if filename.endswith(".jpg"):
		image_path = os.path.join(input_path, filename)
		# Loading a sample image
		img = load_img(image_path)
		# Converting the input sample image to an array
		x = img_to_array(img)
		# Reshaping the input image
		x = x.reshape((1, ) + x.shape)

		# Generating and saving 5 augmented samples
		# using the above defined parameters.
		i = 0
		for batch in datagen.flow(x, batch_size = 16,
								save_to_dir ='C:/Users/Camille/Downloads/NEW PATH',
								save_prefix ='augmented_images', save_format ='jpg'):
			i += 1
			if i > 5:
				break